<a href="https://colab.research.google.com/github/mystify122/PrivacyPolicies/blob/main/Qiskit_QuantumAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Installation


In [ ]:
!pip install qiskit ipywidgets
!pip install qiskit[visualization]
!pip install qiskit_ibm_runtime
!pip install qiskit_aer
!pip install galois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=20f3af1c4a8ee434e044c882cdb950fd673b29dec182352a2bf17e8801af7014
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
     ━━━━━━━━━━━━━

In [ ]:
import qiskit as q
%matplotlib inline

# Deutsch's algorithm - Implementation


First we'll define a quantum circuit that implements a query gate for one of the four functions $ f_1, f_2, f_3$ or $f_4$.

The implementation of query gates is not really a part of Deutsch's algorithm itself — here we're essentially just showing one way to prepare the input (in the form of a circuit implementation of a query gate).

In [ ]:
from qiskit import QuantumCircuit

def deutsch_function(case: int):
    """
    Generate a valid Deutsch function as a `QuantumCircuit`.
    """
    if case not in [1, 2, 3, 4]:
        raise ValueError("`case` must be 1, 2, 3, or 4.")

    f = QuantumCircuit(2)
    if case in [2, 3]:
        f.cx(0, 1)
    if case in [3, 4]:
        f.x(1)
    return f

We can see what each circuit looks like using the `draw` method.

In [ ]:
display(deutsch_function(3).draw())

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

Next we will create the actual quantum circuit for Deutsch's algorithm, substituting the query gate with a quantum circuit implementation given as an argument. Shortly we'll plug in one of the four circuits defined by the function `deutsch_function` we defined earlier. Barriers are included to show the visual separation between the query gate implementation and the rest of the circuit, but they aren't necessary and can safely be removed.

In [ ]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in Deutsch's algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)

    qc.x(n)
    qc.h(range(n + 1))

    qc.barrier()
    qc.compose(function, inplace=True)
    qc.barrier()

    qc.h(range(n))
    qc.measure(range(n), range(n))

    return qc


Again we can see what the circuit looks like using the `draw` method.

In [ ]:
display(compile_circuit(deutsch_function(3)).draw())

┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

Finally, we'll create a function that runs the circuit just defined one time and outputs the appropriate result: "constant" or "balanced."

In [ ]:
from qiskit_aer import AerSimulator

def deutsch_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch function is constant or balanced.
    """
    qc = compile_circuit(function)

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if measurements[0] == "0":
        return "constant"
    return "balanced"

The following code cell runs Deutsch's algorithm on any one of the four functions defined above.



In [ ]:
f = deutsch_function(3)
display(f.draw())
display(deutsch_algorithm(f))

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

'balanced'

# The Deutsch-Jozsa algorithm - Implementation

To implement the Deutsch-Jozsa algorithm in Qiskit, we'll start by generating a quantum circuit that implements a query operation for a randomly selected function that satisfies the promise: with 50% chance the function is constant, and with 50% change the function is balanced. For each possibility, the function is selected uniformly from the possibilities.

The argument to `dj_function` is the number of input bits of the function.

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

def dj_function(num_qubits):
    """
    Create a random Deutsch-Jozsa function.
    """

    qc = QuantumCircuit(num_qubits + 1)
    if np.random.randint(0, 2):
        # Flip output qubit with 50% chance
        qc.x(num_qubits)
    if np.random.randint(0, 2):
        # return constant circuit with 50% chance
        return qc

    # next, choose half the possible input states
    on_states = np.random.choice(
        range(2**num_qubits),  # numbers to sample from
        2**num_qubits // 2,  # number of samples
        replace=False,  # makes sure states are only sampled once
    )

    def add_cx(qc, bit_string):
        for qubit, bit in enumerate(reversed(bit_string)):
            if bit == "1":
                qc.x(qubit)
        return qc

    for state in on_states:
        qc.barrier()  # Barriers are added to help visualize how the functions are created. They can safely be removed.
        qc = add_cx(qc, f"{state:0b}")
        qc.mcx(list(range(num_qubits)), num_qubits)
        qc = add_cx(qc, f"{state:0b}")

    qc.barrier()

    return qc

We can show the quantum circuit implementation of the query gate using the `draw` method as usual.

In [ ]:
display(dj_function(3).draw())

░ ┌───┐     ┌───┐ ░                 ░                 ░ ┌───┐     »
q_0: ──────░─┤ X ├──■──┤ X ├─░────────■────────░────────■────────░─┤ X ├──■──»
           ░ ├───┤  │  ├───┤ ░ ┌───┐  │  ┌───┐ ░ ┌───┐  │  ┌───┐ ░ └───┘  │  »
q_1: ──────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░────────■──»
           ░ └───┘  │  └───┘ ░ ├───┤  │  ├───┤ ░ └───┘  │  └───┘ ░        │  »
q_2: ──────░────────■────────░─┤ X ├──■──┤ X ├─░────────■────────░────────■──»
     ┌───┐ ░      ┌─┴─┐      ░ └───┘┌─┴─┐└───┘ ░      ┌─┴─┐      ░      ┌─┴─┐»
q_3: ┤ X ├─░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├»
     └───┘ ░      └───┘      ░      └───┘      ░      └───┘      ░      └───┘»
«     ┌───┐ ░ 
«q_0: ┤ X ├─░─
«     └───┘ ░ 
«q_1: ──────░─
«           ░ 
«q_2: ──────░─
«           ░ 
«q_3: ──────░─
«           ░

Next we define a function that creates the Deutsch-Jozsa circuit, taking a quantum circuit implementation of a query gate as an argument.

In [ ]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in the Deutsch-Jozsa algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(function, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

Finally, a function that runs the Deutsch-Jozsa circuit once is defined.






In [ ]:
from qiskit_aer import AerSimulator

def dj_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch-Jozsa function is constant or balanced.
    """
    qc = compile_circuit(function)

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if "1" in measurements[0]:
        return "balanced"
    return "constant"

We can test our implementation by choosing a function randomly, displaying the quantum circuit implementation of a query gate for this function, and then running the Deutsch-Jozsa algorithm on that function.



In [ ]:
f = dj_function(3)
display(f.draw())
display(dj_algorithm(f))

░ ┌───┐     ┌───┐ ░ ┌───┐     ┌───┐ ░ ┌───┐     ┌───┐ ░           »
q_0: ──────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░────────■──»
           ░ └───┘  │  └───┘ ░ ├───┤  │  ├───┤ ░ └───┘  │  └───┘ ░        │  »
q_1: ──────░────────■────────░─┤ X ├──■──┤ X ├─░────────■────────░────────■──»
           ░ ┌───┐  │  ┌───┐ ░ └───┘  │  └───┘ ░        │        ░ ┌───┐  │  »
q_2: ──────░─┤ X ├──■──┤ X ├─░────────■────────░────────■────────░─┤ X ├──■──»
     ┌───┐ ░ └───┘┌─┴─┐└───┘ ░      ┌─┴─┐      ░      ┌─┴─┐      ░ └───┘┌─┴─┐»
q_3: ┤ X ├─░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├»
     └───┘ ░      └───┘      ░      └───┘      ░      └───┘      ░      └───┘»
«           ░ 
«q_0: ──────░─
«           ░ 
«q_1: ──────░─
«     ┌───┐ ░ 
«q_2: ┤ X ├─░─
«     └───┘ ░ 
«q_3: ──────░─
«           ░

'balanced'

# The Bernstein-Vazirani problem - Implementation


We've already implemented the Deutsch-Jozsa circuit above, and here we will make use of it to solve the Bernstein-Vazirani problem. First we'll define a function that implements a query gate for the Bernstein-Vazirani problem given any binary string $s$.

In [ ]:
def bv_function(s):
    """
    Create a Bernstein-Vazirani function from a string of 1s and 0s.
    """
    qc = QuantumCircuit(len(s) + 1)
    for index, bit in enumerate(reversed(s)):
        if bit == "1":
            qc.cx(index, len(s))
    return qc

display(bv_function("1001").draw())

q_0: ──■───────
       │       
q_1: ──┼───────
       │       
q_2: ──┼───────
       │       
q_3: ──┼────■──
     ┌─┴─┐┌─┴─┐
q_4: ┤ X ├┤ X ├
     └───┘└───┘

Now we can run the Deutsch-Jozsa circuit on the function. Note that the `compile_circuit` function was defined in a previous code cell, which must be run prior to the cell that follows.

In [ ]:
def bv_algorithm(function: QuantumCircuit):
    qc = compile_circuit(function)
    result = AerSimulator().run(qc, shots=1, memory=True).result()
    return result.get_memory()[0]

display(bv_algorithm(bv_function("1001")))

'1001'

# Simon's Algorithm



To implement Simon's algorithm in Qiskit, we'll use the fact that we can convert unitary matrices into gates in Qiskit using the `.unitary` method. Specifically, we'll use this methodology to define a query gate for a randomly chosen function satisfying Simon's problem for a given string $s$.

In [ ]:
# import random
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit
import numpy as np

def simon_function(s: str):
    """
    Create a QuantumCircuit implementing a query gate for Simon problem obeying the promise for the hidden string `s`
    """
    # Our quantum circuit has 2n qubits for n = len(s)
    n = len(s)
    qc = QuantumCircuit(2 * n)

    # Define a random permutation of all n bit strings. This permutation will effectively hide the string s.
    pi = np.random.permutation(2**n)

    # Now we'll define a query gate explicitly. The idea is to first define a function g(x) = min{x,x ^ s}, which
    # is a simple function that satisfies the promise, and then we take f to be the composition of g and the random
    # permutation pi. This gives us a random function satisfying the promise for s.

    query_gate = np.zeros((4**n, 4**n))
    for x in range(2**n):
        for y in range(2**n):
            z = y ^ pi[min(x, x ^ int(s, 2))]
            query_gate[x + 2**n * z, x + 2**n * y] = 1

    # Our circuit has just this one query gate
    qc.unitary(query_gate, range(2 * n))
    return qc

Next we'll define a function that runs the circuit in Simon's problem $k$ times and reports the results.

In [ ]:
from qiskit_aer import AerSimulator
from qiskit import ClassicalRegister

def simon_measurements(problem: QuantumCircuit, k: int):
    """
    Quantum part of Simon's algorithm. Given a `QuantumCircuit` that
    implements f, get `k` measurements to be post-processed later.
    """
    n = problem.num_qubits // 2

    qc = QuantumCircuit(2 * n, n)
    qc.h(range(n))
    qc.compose(problem, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))

    result = AerSimulator().run(qc, shots=k, memory=True).result()
    return result.get_memory()

The following code cell illustrates how the function works when we plug in the query gate constructed above. Feel free to try different arguments, but keep in mind that the cost of the simulation we've built is exponential in the number of qubits we require — so don't make the string $s$ too long if you don't want to wait!

In [ ]:
display(simon_measurements(simon_function("11011"),k=12))

['10110',
 '10101',
 '00011',
 '10110',
 '10110',
 '10010',
 '10001',
 '10001',
 '00011',
 '11111',
 '00100',
 '00100']

To do the post-processing, we can make use of the `galois` package, which has a built-in function for computing the null space modulo 2.

In [ ]:
import numpy as np
import galois

def simon_algorithm(problem: QuantumCircuit):
    """
    Given a `QuantumCircuit` that implements a query gate for Simon problem, return the hidden string `s`.
    """

    # Quantum part: run the circuit defined previously k times and gather the measurement results.
    # Replace +10 by +r for any nonnegative integer r depending on desired confidence.

    measurements = simon_measurements(problem, k=problem.num_qubits // 2 + 10)
    print("Measurement results:")
    display(measurements)

    # Classical post-processing:

    # 1. Convert measurements of form '11101' to 2D-array of integers
    matrix = np.array([list(bitstring) for bitstring in measurements]).astype(int)

    # 2. Interpret matrix as using arithmetic mod 2, and find null space
    null_space = galois.GF(2)(matrix).null_space()
    print("Null space:")
    display(null_space)

    # 3. Convert back to a string
    print("Guess for hidden string s:")
    if len(null_space) == 0:
        # No non-trivial solution; `s` is all-zeros
        return "0" * len(measurements[0])
    return "".join(np.array(null_space[0]).astype(str))

And finally we can try it out.



In [ ]:
display(simon_algorithm(simon_function("10011")))

Measurement results:


['10110',
 '00111',
 '10101',
 '00000',
 '01100',
 '01111',
 '10001',
 '10001',
 '10101',
 '00111',
 '10110',
 '01000',
 '01111',
 '01111',
 '10110']

Null space:


GF([[1, 0, 0, 1, 1]], order=2)

Guess for hidden string s:


'10011'